# quake 3


## play quake 3

TODO: drop in to quake 3 browser with server backend. configured. shared. hosted. offer to invite friends with a link.




### use vnc as a backup


### clone of https://openarena.live/

or http://www.quakejs.com/play?set%20fs_game%20cpma&set%20mode_start%20FFA&set%20g_teamAutoJoin%201&map%20cpm1a
